In [1]:
library(NLP)
library(tm)


Attaching package: ‘NLP’


The following object is masked from ‘package:httr’:

    content




In [2]:
library(e1071)
library(tidyverse)
library(RColorBrewer)
library(wordcloud)
library(ggplot2)  
library(beeswarm)
library(quantmod)
library(reshape2)
library(plyr)
library(scales)
library(viridis)
library(zoo)
library(dplyr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ ggplot2::annotate() masks NLP::annotate()
✖ NLP::content()      masks httr::content()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()

Loading required package: xts

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘xts’


The following objects are masked from ‘package:dplyr’:

    first, last


Loading required package: TTR

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Version 0.4-0 included new data defaults. See ?getSymbols.


Attaching package: ‘reshape2’


The following obj

In [3]:
list.files(path = "../input")

[1] "sms-spam-collection-dataset"

In [4]:
DATA1=read_csv('../input/sms-spam-collection-dataset/spam.csv')

Warning message:
“Missing column names filled in: 'X3' [3], 'X4' [4], 'X5' [5]”

── Column specification ────────────────────────────────────────────────────────
cols(
  v1 = col_character(),
  v2 = col_character(),
  X3 = col_character(),
  X4 = col_character(),
  X5 = col_character()
)




In [5]:
head(DATA1)

Warning message in FUN(X[[i]], ...):
“input string 2 is invalid in this locale”
Warning message in FUN(X[[i]], ...):
“input string 2 is invalid in this locale”
Warning message in FUN(X[[i]], ...):
“input string 2 is invalid in this locale”
ERROR while rich displaying an object: Error in gsub(chr, html_specials[[chr]], text, fixed = TRUE): input string 2 is invalid in this locale

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         st

v1,v2,X3,X4,X5
<chr>,<chr>,<chr>,<chr>,<chr>
ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",NA,NA,NA
ham,Ok lar... Joking wif u oni...,NA,NA,NA
spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,NA,NA,NA
ham,U dun say so early hor... U c already then say...,NA,NA,NA
ham,"Nah I don't think he goes to usf, he lives around here though",NA,NA,NA
spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, 1.50 to rcv",NA,NA,NA


In [6]:
data=DATA1[0:2]

In [7]:
data <- sapply(data,function(row) iconv(row, "latin1", "ASCII", sub=""))
data <- data[,1:2]
data <- data.frame(data)
names(data) <- c('type','text')
data$type <- factor(data$type)
table(data$type)


 ham spam 
4825  747 

In [8]:
data

type,text
<fct>,<fct>
ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
ham,Ok lar... Joking wif u oni...
spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham,U dun say so early hor... U c already then say...
ham,"Nah I don't think he goes to usf, he lives around here though"
spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, 1.50 to rcv"
ham,Even my brother is not like to speak with me. They treat me like aids patent.
ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam,WINNER!! As a valued network customer you have been selected to receivea 900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.


In [9]:
bind1=Corpus(VectorSource(data$text))



In [10]:
bind1=tm_map(bind1,content_transformer(tolower))


Warning message in tm_map.SimpleCorpus(bind1, content_transformer(tolower)):
“transformation drops documents”


In [11]:
bind1=tm_map(bind1,content_transformer(removeNumbers))


Warning message in tm_map.SimpleCorpus(bind1, content_transformer(removeNumbers)):
“transformation drops documents”


In [12]:
bind1=tm_map(bind1,removeWords,stopwords())


Warning message in tm_map.SimpleCorpus(bind1, removeWords, stopwords()):
“transformation drops documents”


In [13]:
bind1=tm_map(bind1,content_transformer(removePunctuation))


Warning message in tm_map.SimpleCorpus(bind1, content_transformer(removePunctuation)):
“transformation drops documents”


In [14]:
bind1=tm_map(bind1,content_transformer(stripWhitespace))

Warning message in tm_map.SimpleCorpus(bind1, content_transformer(stripWhitespace)):
“transformation drops documents”


In [15]:
bind1

<<SimpleCorpus>>
Metadata:  corpus specific: 1, document level (indexed): 0
Content:  documents: 5572

In [16]:
matrid=DocumentTermMatrix(bind1)

In [17]:
ham=subset(bind1,data$type=="ham")
spam=subset(bind1,data$type=="spam")

In [18]:
step1_train=data[1:4169,]$type
step1_test=data[4169:5572,]$type





In [19]:
step2_train=matrid[1:4169,]
step2_test=matrid[4169:5572,]



In [20]:
step3_train=bind1[1:4169]
step3_test=bind1[4169:5572]

In [21]:
f_data=findFreqTerms(matrid,5)

In [22]:
step4_train=step2_train[,f_data]
step4_test=step2_test[,f_data]

In [23]:
yes_no =function(x)
{y=ifelse(x>0 ,1,0)
y=factor(x,levels = c(0,1),labels = c("yes","no"))
y}

In [24]:
step5_train=apply(step4_train,2,yes_no)
step5_test=apply(step4_test,2,yes_no)



In [25]:
step5_train

,available,bugis,cine,crazy,got,great,point,wat,world,joking,⋯,ignore,tariffs,tick,thurs,planned,relation,lik,asks,meds,common
1,no,no,no,no,no,no,no,no,no,yes,⋯,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
2,yes,yes,yes,yes,yes,yes,yes,yes,yes,no,⋯,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
3,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,⋯,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
4,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,⋯,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
5,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,⋯,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
6,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,⋯,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
7,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,⋯,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
8,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,⋯,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
9,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,⋯,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
10,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,⋯,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes


In [26]:
step1_train

[1] ham  ham  spam ham  ham  spam ham  ham  spam spam ham  spam spam ham 
  [15] ham  spam ham  ham  ham  spam ham  ham  ham  ham  ham  ham  ham  ham 
  [29] ham  ham  ham  ham  ham  ham  spam ham  ham  ham  ham  ham  ham  ham 
  [43] spam ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  spam ham 
  [57] spam ham  ham  ham  ham  ham  ham  ham  ham  spam ham  spam spam ham 
  [71] ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham 
  [85] ham  ham  ham  ham  ham  ham  ham  ham  ham  spam ham  spam ham  ham 
  [99] ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham 
 [113] ham  spam ham  ham  spam ham  ham  spam spam ham  spam ham  ham  ham 
 [127] ham  ham  ham  ham  ham  ham  ham  spam spam ham  ham  ham  spam ham 
 [141] ham  ham  ham  ham  ham  ham  spam ham  ham  ham  ham  ham  ham  ham 
 [155] ham  ham  ham  ham  spam spam ham  ham  ham  spam spam ham  spam ham 
 [169] ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham 
 [183] ham  ham  ham  ham  ham  spam ham  ham  spam ham  ham  ham  ham  ham 
 [197] ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham 
 [211] ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham 
 [225] spam ham  spam ham  ham  ham  ham  ham  ham  ham  spam ham  ham  ham 
 [239] ham  spam ham  ham  ham  ham  ham  ham  ham  ham  ham  spam ham  ham 
 [253] ham  ham  ham  ham  ham  ham  spam ham  ham  ham  ham  spam ham  ham 
 [267] ham  spam ham  spam ham  ham  spam ham  ham  ham  ham  ham  ham  ham 
 [281] ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham 
 [295] ham  spam ham  ham  ham  ham  ham  ham  ham  ham  spam ham  ham  ham 
 [309] spam ham  ham  spam ham  ham  ham  ham  ham  ham  spam ham  ham  ham 
 [323] ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  spam ham  spam ham 
 [337] ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  spam ham 
 [351] ham  ham  ham  ham  ham  ham  spam spam ham  ham  ham  ham  ham  ham 
 [365] ham  ham  spam spam ham  ham  ham  ham  ham  ham  spam ham  ham  ham 
 [379] ham  ham  ham  ham  ham  ham  spam ham  ham  ham  spam ham  ham  ham 
 [393] ham  ham  ham  ham  ham  ham  ham  ham  spam ham  ham  ham  ham  ham 
 [407] ham  ham  ham  ham  ham  ham  ham  ham  spam ham  ham  spam ham  spam
 [421] ham  spam ham  spam ham  ham  ham  ham  ham  ham  ham  ham  spam ham 
 [435] ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham 
 [449] ham  ham  ham  ham  ham  ham  spam ham  ham  ham  ham  ham  ham  ham 
 [463] spam ham  ham  ham  ham  ham  ham  ham  spam ham  ham  spam ham  ham 
 [477] ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  spam ham  ham  ham 
 [491] ham  spam ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham 
 [505] spam ham  ham  ham  ham  ham  ham  ham  ham  ham  spam ham  spam spam
 [519] ham  ham  ham  ham  ham  ham  spam ham  spam ham  spam ham  spam ham 
 [533] ham  ham  ham  ham  ham  ham  ham  ham  spam ham  ham  ham  ham  ham 
 [547] ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham 
 [561] ham  ham  ham  spam ham  ham  ham  ham  ham  ham  ham  ham  ham  ham 
 [575] ham  spam ham  ham  spam ham  ham  ham  spam ham  ham  ham  ham  ham 
 [589] ham  ham  spam spam spam ham  ham  ham  ham  spam ham  ham  ham  ham 
 [603] ham  ham  ham  ham  spam ham  ham  ham  spam ham  ham  ham  ham  ham 
 [617] ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  spam spam
 [631] ham  ham  ham  ham  spam ham  ham  ham  ham  ham  ham  ham  ham  ham 
 [645] ham  ham  ham  spam ham  spam ham  ham  ham  ham  ham  ham  ham  ham 
 [659] ham  spam ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  spam
 [673] spam ham  ham  ham  ham  ham  ham  ham  ham  ham  ham  spam ham  ham 
 [687] ham  ham  ham  spam ham  ham  ham  ham  ham  ham  ham  ham  ham  ham 
 [701] ham  ham  ham  ham  ham  ham  ham  ham  spam spam ham  ham  spam ham 
 [715] ham  ham  spam ham  spam ham  ham  ham  ham  ham  ham  ham  ham  ham 

In [27]:
model=naiveBayes(step5_train,step1_train)


In [28]:
pred_val=predict(model,step5_test)


In [29]:
con_matrix=table(step1_test,pred_val)

In [30]:
con_matrix

          pred_val
step1_test  ham spam
      ham  1215    7
      spam   26  156

In [31]:
Accuracy =sum(diag(con_matrix))/sum(con_matrix)*100

In [32]:
Accuracy

[1] 97.64957